In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout,SimpleRNN
import warnings
warnings.filterwarnings('ignore')

In [3]:
max_feature = 10000 #voc size
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=max_feature)
print(f'X Train Shape: {X_train.shape}')
print(f'X Test Shape: {X_test.shape}')
print(f'Y Train Shape: {y_train.shape}')
print(f'Y Test Shape: {y_test.shape}')

X Train Shape: (25000,)
X Test Shape: (25000,)
Y Train Shape: (25000,)
Y Test Shape: (25000,)


In [4]:
## INSEPECT SAMPLE REVIEW AND DATAASET
smaple_review = X_train[0]
sample_label = y_train[0]
print(f'Sample Review As (Integretaion): {smaple_review}')
print(f'Sample Label As (Integretaion): {sample_label}')

Sample Review As (Integretaion): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample Label As (Integretai

In [5]:
word_index = imdb.get_word_index()
# word_index

In [6]:
Reverse_word = {value:key for key,value in word_index.items()}
# Reverse_word

In [7]:
decode_review = ' '.join(Reverse_word.get(i-3, '?') for i in smaple_review)
decode_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [8]:
max_len = 500
X_train = sequence.pad_sequences(X_train,maxlen=max_len)
X_test = sequence.pad_sequences(X_test,maxlen=max_len)

In [9]:
max_features = 10000
model=Sequential()
model.add(Embedding(max_features,128,input_length=max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation="sigmoid"))
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
earlystoping = EarlyStopping(monitor = 'val_loss',patience=5,restore_best_weights=True)

model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['Accuracy']
             )

In [11]:
history = model.fit(X_train,y_train, epochs=10,batch_size=32,
         validation_split=0.2,
         callbacks=[earlystoping]
        )

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 206ms/step - Accuracy: 0.5874 - loss: 8.8671 - val_Accuracy: 0.7780 - val_loss: 0.4599
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 205ms/step - Accuracy: 0.7414 - loss: 7.4868 - val_Accuracy: 0.7552 - val_loss: 0.5195
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 126s 202ms/step - Accuracy: 0.7631 - loss: 779123.5000 - val_Accuracy: 0.6554 - val_loss: 0.5903
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 145s 207ms/step - Accuracy: 0.7079 - loss: 0.5938 - val_Accuracy: 0.6158 - val_loss: 0.6186
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 126s 202ms/step - Accuracy: 0.7077 - loss: 0.5510 - val_Accuracy: 0.6310 - val_loss: 0.6123
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 207ms/step - Accuracy: 0.7340 - loss: 0.5224 - val_Accuracy: 0.6882 - val_loss: 0.5628


In [17]:
model.save('RNN_model.h5')

In [18]:
from tensorflow.keras.models import load_model
my_model = load_model('/content/RNN_model.h5')

In [19]:
def decode_review(encoded_review):
    return ' '.join(Reverse_word.get(i-3, '?') for i in encoded_review)

def preprocess_text(text):
  word = text.lower().split()
  encode_review = [word_index.get(word,2)+3 for word in word]
  return sequence.pad_sequences([encode_review],maxlen=max_len)



In [20]:
def predict_sentiment(text):
  encoded_review = preprocess_text(text)
  prediction = my_model.predict(encoded_review)[0][0]
  print(prediction)
  if prediction > 0.5:
    return 'Positive'
  else:
    return 'Negative'

In [21]:
sample_review = 'This movie was fantastic! The acting was great and the plot was thrilling'
sentiment = predict_sentiment(sample_review)
print(f'Sentiment: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
0.77057105
Sentiment: Positive
